In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

- Create mesh 
    - Depends on element size and extents
    - Returns coordinate axes (size of each is nx, ny, nz)
- Add structures to mesh
    - Get nodes in struct based on coordinates
    - Change part id of elements in struct
        - All nodes in element should be within struct? 
    -- Generate list of node ids in struct
- Add boundary constraints 
    - Depends on mesh symmetry
    - Change tc and rc of nodes on faces and edges
- Add pml
    - Depends on mesh symmetry and pml thickness
    - Change part id of non-symmetry plane elements
    - Part id + 1 if also in struct
- Finish dyna deck
    - Add material
    - Add load curves
    - Add arf load based on field params (pre-generated)
    - Add database/control parameters
        - Node sets?
- Validation functions
    - Make sure nodes exist on center axis regardless of mesh symmetry so peak arf load is modeled

- Check get_plane_node_ids to see if 0s are necessary
- Fix constraints in constrain_boundary_nodes
- Fix formatting of numbers in material cards
- Fix pml/struct interface with 4 node overlap 
    - Check len(pml_and_struct_nodes) condition is valid
- Add validation functions
    - Nodes exist on center axis regardless of mesh symmetry so peak arf load is modeled
    - Mesh extent minima are less than maxima

In [ ]:
# @dataclass
# class Extent:
#     x: Tuple[float, float]
#     y: Tuple[float, float]
#     z: Tuple[float, float]

# @dataclass
# class Coordinates:
#     nx: int
#     ny: int
#     nz: int
#     extent: Extent
#     x: np.ndarray = field(init=False, repr=False)
#     y: np.ndarray = field(init=False, repr=False)
#     z: np.ndarray = field(init=False, repr=False)

#     def __post_init__(self):
#         self.x = np.linspace(self.extent.x[0], self.extent.x[1], self.nx)
#         self.y = np.linspace(self.extent.y[0], self.extent.y[1], self.ny)
#         self.z = np.linspace(self.extent.z[0], self.extent.z[1], self.nz)


In [3]:
from fem.dyna.mesh import Coordinates, DynaMesh
from fem.dyna.material import KelvinMaxwellViscoelastic

mat = KelvinMaxwellViscoelastic(density=1, E=26.11, nu=0.499, eta=2.34)
mat2 = KelvinMaxwellViscoelastic(density=1, E=2*26.11, nu=0.499, eta=2.34)

struct_list = [
    {
        'shape': 'cube', 
        'struct_opts': [-1, -0.5, -0.5, -0, -4, -2],
        'material': mat2,  
    }, 
    {
        'shape': 'sphere', 
        'struct_opts': [0, 0, -2, 1],
        'material': mat2,  
    }, 
]

coords = Coordinates(
    nx=11, ny=11, nz=11,
    xmin=-1.0, xmax=0.0,
    ymin=-1.0, ymax=0.0,
    zmin=-4.0, zmax=0.0
)
symmetry = 'q'
mesh = DynaMesh(coords, symmetry, mat)
mesh.constrain_boundary_nodes()
mesh.add_pml(pml_thickness=2)

for struct in struct_list:
    mesh.add_struct(**struct)

# print(mesh)
print(mesh.material_card_string)


*MAT_KELVIN-MAXWELL_VISCOELASTIC
$ -- Youngs = 26.11 kPa, Viscosity = 2.34 Pa.s, Tau = 89.62 ms, Poisson = 0.49900
$ MID, DENSITY, K, G0, Gi, dc, f0, s0
1, 1.000000, 4.35166667E+04, 17418.279, 87.091394, 4.48104175E-04, 1.0000000, 0.0000000
*PART
$ Title
background
$      pid     secid       mid     eosid      hgid      grav    adpopt      tmid
         1         1         1         0         0         0         0         0
*SECTION_SOLID
$    secid    elform       aet
         1         2         0

*MAT_PML_ELASTICC
$ MID, DENSITY, E, NU
2, 1.0000000, 26.110000, 0.49900000
*PART
$ Title
background pml
$      pid     secid       mid     eosid      hgid      grav    adpopt      tmid
         2         2         2         0         0         0         0         0
*SECTION_SOLID
$    secid    elform       aet
         2         2         0

*MAT_KELVIN-MAXWELL_VISCOELASTIC
$ -- Youngs = 52.22 kPa, Viscosity = 2.34 Pa.s, Tau = 44.81 ms, Poisson = 0.49900
$ MID, DENSITY, K, G0, Gi, dc, f0,

In [ ]:
np.unique(mesh.elems['pid'])